In [ ]:
!git clone https://github.com/WorkInTheDark/FairytaleQA_QAG_System.git
%cd ./FairytaleQA_QAG_System/

Save Ranking Model into PATH_TO_RANK

In [ ]:
!pip install datasets
!pip install tqdm==4.43.0
!pip install transformers

In [ ]:
from datasets import load_dataset
from pathlib import Path

In [ ]:
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    learning_rate=1.541663609697611e-05,
    num_train_epochs=4,       # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=30000,
    save_steps=100000,
    seed=15,

)

model = DistilBertForSequenceClassification.from_pretrained("PATH_TO_RANK/RANKING_QA_DistilBert_QAC_lr1e-5_e4_seed15_b1_uneven_distributed", local_files_only=True)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    # train_dataset=train_dataset,         # training dataset
    # eval_dataset=val_dataset             # evaluation dataset
)

In [ ]:
import torch

class RankingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


test data

In [ ]:
!mkdir ./FairytaleQA_QAG_System/result
!mkdir ./FairytaleQA_QAG_System/result/test

In [ ]:
save_directory = "./FairytaleQA_QAG_System/result/test"

In [ ]:
import glob 

generation_results = []

total_generated_results = 0


cnt_book = 0

book_directory = glob.glob("./FairytaleQA_QAG_System/data/generated_qa/test/*.txt", recursive=True)


for each_book in book_directory:
  key = each_book.split('/')[-1].replace('.txt', '').replace('generated_qa_stories-', '').lower()

  # this to be a section-key dictionary
  generation_answers = []
  generation_data_for_ranking = []

  lines_generate = open(each_book, 'r').readlines()
  
  ## for i in lines_generate:
  for i in lines_generate:
    # each line should be A </s> C </s> Q
    
    generation_answers.append( i.replace('\n', '')  )
    
    question = i.replace('\n', '').split('</s>')[-1].strip()
    answer = i.replace('\n', '').split('</s>')[0].strip()
    section = i.replace('\n', '').split('</s>')[1].strip()

    # Q + A + C
    generation_data_for_ranking.append( question + ' <SEP> ' + answer  + ' <SEP> ' + section)


    total_generated_results += 1



  # create the evaluation dataset for the ranking model, and run through the model
  
  test_encodings = tokenizer(generation_data_for_ranking, truncation=True, padding=True)
  test_labels = [1]*len(generation_data_for_ranking)
  test_dataset = RankingDataset(test_encodings, test_labels)

  predictions = trainer.predict(test_dataset)
  
  assert len(predictions.predictions) == len(generation_answers)

  # Here you may store all ranked QA-pairs along with the score into a file
  # Or just directly sort by the values and select the top-N ones

  
  f1 = open(save_directory + '/ranked_generated_qa_stories-' + key + ".txt", 'w')
  
  for i in range(len(predictions.predictions)):
    predicted =  predictions.predictions[i][1]
    f1.write( generation_answers[i] + '<SEP>' + str(predicted) + '\n' )
  
  f1.close()


  cnt_book += 1

val data

In [ ]:
!mkdir ./FairytaleQA_QAG_System/result/val

In [ ]:
save_directory = "./FairytaleQA_QAG_System/result/val"

In [ ]:
import glob 

generation_results = []

total_generated_results = 0


cnt_book = 0

book_directory = glob.glob("./FairytaleQA_QAG_System/data/generated_qa/val/*.txt", recursive=True)


# for key in val_name_set:
# for key in test_name_set:
for each_book in book_directory:
  key = each_book.split('/')[-1].replace('.txt', '').replace('generated_qa_stories-', '').lower()

  # this to be a section-key dictionary
  generation_answers = []
  generation_data_for_ranking = []

  lines_generate = open(each_book, 'r').readlines()

  for i in lines_generate:

    generation_answers.append( i.replace('\n', '')  )
    
    question = i.replace('\n', '').split('</s>')[-1].strip()
    answer = i.replace('\n', '').split('</s>')[0].strip()
    section = i.replace('\n', '').split('</s>')[1].strip()


    # Q + A + C
    generation_data_for_ranking.append( question + ' <SEP> ' + answer + ' <SEP> ' + section)

    total_generated_results += 1



  # create the evaluation dataset for the ranking model, and run through the model
  
  test_encodings = tokenizer(generation_data_for_ranking, truncation=True, padding=True)
  test_labels = [1]*len(generation_data_for_ranking)
  test_dataset = RankingDataset(test_encodings, test_labels)

  predictions = trainer.predict(test_dataset)
  
  assert len(predictions.predictions) == len(generation_answers)

  # Here you may store all ranked QA-pairs along with the score into a file
  # Or just directly sort by the values and select the top-N ones

  
  f1 = open(save_directory + '/ranked_generated_qa_stories-' + key + ".txt", 'w')
  
  for i in range(len(predictions.predictions)):
    predicted =  predictions.predictions[i][1]
    f1.write( generation_answers[i] + '<SEP>' + str(predicted) + '\n' )
  
  f1.close()


  cnt_book += 1